# D-DUST Data Grid Processing Notebook

Check the following link to Data Management Plan and the Variable list table: <br>
**1. [Data Management Plan (DMP)](https://docs.google.com/document/d/1n3PVat7PBTG76JnINOkL2pvBZuKQlakZkTgqNj39oAQ/edit#)**<br>
**2. [Variables list table](https://docs.google.com/spreadsheets/d/1-5pwMSc1QlFyC8iIaA-l1fWhWtpqVio2/edit#gid=91313358)**

This notebook describes the physical variables selected for the project and how they are preprocessed.
These variables are divided into <u>4 categories</u>, as shown in the Variable list table:
1. **Map Layer**: static layer used to describe Lombardy region morphology and its features (such as elevation, infrastructures, land use and cover etc.)
2. **Model**: data retrieved from a model that uses satellite and in-situ observations of meteorological and air quality data as input (such as ERA5, CAMS).
3. **Satellite**: data obtained directly from satellite observations (such as Sentinel-5P).
4. **Ground Sensor**: data retrieved from ground monitoring stations measuring air quality and meteorological variables.

## Import libraries

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rasterstats as rstat
import rioxarray
import shapely.speedups
import matplotlib.pyplot as plt
from shapely.geometry import shape
from shapely.geometry import  MultiLineString
shapely.speedups.enable()

In [ ]:
absolutepath = os.path.dirname(os.path.abspath("__file__"))

In [ ]:
print(absolutepath)

## Import grids

Three grids with different spatial resolution are used in this project:
1. **grid_cams**: 0.1° x 0.1° resolution - Grid with CAMS Model spatial resolution.
2. **grid_s5p**: 0.066° x 0.066° resolution - Grid with the Sentinel-5P approximate spatial resolution.
3. **grid_st**: 0.01° x 0.01° resolution- Grid generated with at most one ARPA monitoring station for each pixel.

These grids are defined as bounding box of the Lombardy region layer applying a buffer of 20 km.

In [ ]:
grid_cams_path = absolutepath + '/grid/grid0_1.gpkg'
grid_s5p_path = absolutepath + '/grid/grid0_066.gpkg'
grid_st_path = absolutepath + '/grid/grid0_01.gpkg'
grid_cams = gpd.read_file(grid_cams_path)
grid_s5p = gpd.read_file(grid_s5p_path)
grid_st = gpd.read_file(grid_st_path)

Grid selection

In [ ]:
grid = grid_cams

In [ ]:
m_to_km = 10**(-3)
m2_to_km2 = 10**(-6)

---

# Importing Map Layers

### [DUSAF - Land use - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B18EE7CDC-E51B-4DFB-99F8-3CF416FC3C70%7D) <br>

Consists in a multi-temporal geographic database that classifies land based on major land cover and land use types. Reference system EPSG:4326.<br>
Land use:
- 2 = Aree agricole.
- 3 =Territori boscati e ambienti seminaturali.
- 4 = Aree umide.
- 5 = Corpi idrici.
- 11 = Zone urbanizzate.
- 12 = Insediamenti produttivo, grandi impianti e reti di comunicazione.
- 13 = Aree estrattive, discariche, cantieri, terreni artefatti e abbandonati.
- 14 = Aree verdi non agricole.

In [ ]:
dusaf_path = absolutepath + '/land_use_cover/DUSAF6_dissolve_rast_4326.tiff'

In [ ]:
dusaf = rio.open(dusaf_path)
dusaf_array = dusaf.read(1).astype('float64') 
dusaf_array[dusaf_array<1.0]=np.nan
affine = dusaf.transform

In [ ]:
stats = rstat.zonal_stats(grid, dusaf_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "dusaf"})

In [ ]:
# Class counts in each tile
stats = rstat.zonal_stats(grid, dusaf_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats, orient='columns')
p = p*m2_to_km2

grid['dsf2'] = p[2.0]
grid['dsf3'] = p[3.0]
grid['dsf4'] = p[4.0]
grid['dsf5'] = p[5.0]
grid['dsf11'] = p[11.0]
grid['dsf12'] = p[12.0]
grid['dsf13'] = p[13.0]
grid['dsf14'] = p[14.0]
grid['dsfSum'] = p['count']

 - - -

### [SIARL - Agricultural use - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B83483117-8742-4A1F-A16E-3A48AEE2EBE2%7D) <br>
This layer contains the agricoltural use for each cadastral parcel provided by SIARL 2019 Catalog for the Lombardy region. Reference system EPSG:4326. <br>
Agricoltural use:
1. Altre coltivazioni agrarie
2. Altri cereali
3. Barbabietola
4. Boschi e colture arboree
5. Coltivazioni florovivaistiche
6. Coltivazioni orticole
7. Foraggere
8. Frutticole
9. Mais
10. Olivo
11. Piante industriali e legumi secchi
12. Riso
13. Sementi
14. Tare e incolti
15.  Terreni a riposo
16. Vite
17. Aree antropizzate
18. Aree sterili naturali
19. Corpi idrici
20. Terreni agricoli non classificabili
21. Vegetazione naturale

In [ ]:
siarl_path = absolutepath + '/land_use_cover/siarl.tif'
siarl = rio.open(siarl_path)

In [ ]:
siarl_array = siarl.read(1).astype('float64') 
siarl_array[siarl_array<1.0]=np.nan
affine = siarl.transform

In [ ]:
stats_siarl = rstat.zonal_stats(grid, siarl_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats_siarl]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "siarl"})

In [ ]:
# Class counts in each tile
stats_siarl = rstat.zonal_stats(grid, siarl_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats_siarl, orient='columns')
p = p*m2_to_km2
grid['siarl1'] = p[1.0]
grid['siarl2'] = p[2.0]
grid['siarl3'] = p[3.0]
grid['siarl4'] = p[4.0]
grid['siarl5'] = p[5.0]
grid['siarl6'] = p[6.0]
grid['siarl7'] = p[7.0]
grid['siarl8'] = p[8.0]
grid['siarl9'] = p[9.0]
grid['siarl10'] = p[10.0]
grid['siarl11'] = p[11.0]
grid['siarl12'] = p[12.0]
grid['siarl13'] = p[13.0]
grid['siarl14'] = p[14.0]
grid['siarl15'] = p[15.0]
grid['siarl16'] = p[16.0]
grid['siarl17'] = p[17.0]
grid['siarl18'] = p[18.0]
grid['siarl19'] = p[19.0]
grid['siarl20'] = p[20.0]
grid['siarl21'] = p[21.0]

grid['siarlSum'] = p['count']

- - -

### [Digital Terrain Model - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7BFC06681A-2403-481F-B6FE-5F952DD48BAF%7D)<br>
Digital Terrain Model of Lombardy region with 20 m resolution. Reference system EPSG:4326.
1. **Elevation** = DTM with 20 m resolution
2. **Aspect** = calculated previously from the elevation layer
3. **Slope** = calculated previously from the elevation layer

In [ ]:
dtm_path = absolutepath + '/terrain/dtm20.tif'
aspect_path = absolutepath + '/terrain/aspect.tif'
slope_path = absolutepath + '/terrain/slope.tif'
dtm = rio.open(dtm_path)
aspect = rio.open(aspect_path)
slope = rio.open(slope_path)

In [ ]:
dtm_array = dtm.read(1)
dtm_array[dtm_array<0]=np.nan
affine = dtm.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, dtm_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "h_mean"})

In [ ]:
aspect_array = aspect.read(1)
aspect_array[aspect_array<0]=np.nan
affine = aspect.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aspect_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aspect_mean"})

In [ ]:
slope_array = slope.read(1)
slope_array[slope_array<0]=np.nan
affine = slope.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, slope_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "slope_mean"})

- - -

### Import climate:

In [ ]:
# Set paths for temperature at 2m level, precipitation and ndvi
temp_2m_path = absolutepath + '/temp/temp_2m.tif'
prec_path = absolutepath + '/temp/prec.tif'
press_path = absolutepath + '/temp/press.tif'
ndvi_path = absolutepath + '/temp/ndvi.tif'
n_wind_path = absolutepath + '/temp/n_wind.tif'
e_wind_path = absolutepath + '/temp/e_wind.tif'
soil_moist_path = absolutepath + '/temp/soil_hum.tif'

temp_2m = rio.open(temp_2m_path)
prec = rio.open(prec_path)
press = rio.open(press_path)
ndvi = rio.open(ndvi_path)
n_wind = rio.open(n_wind_path)
e_wind = rio.open(e_wind_path)
soil_moist = rio.open(soil_moist_path)

In [ ]:
temp_2m_array = temp_2m.read(1)
affine = temp_2m.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, temp_2m_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "temp_2m"})

In [ ]:
prec_array = prec.read(1)
prec_array[prec_array<0]=np.nan
affine = prec.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, prec_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "prec"})

In [ ]:
press_array = press.read(1)
press_array[press_array<0]=np.nan
affine = press.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, press_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "press"})

In [ ]:
ndvi_array = ndvi.read(1)
ndvi_array[ndvi_array<0]=np.nan
affine = ndvi.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, ndvi_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ndvi"})

In [ ]:
n_wind_array = n_wind.read(1)
affine = n_wind.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, n_wind_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "n_wind"})

In [ ]:
e_wind_array = e_wind.read(1)
affine = e_wind.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, e_wind_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "e_wind"})

In [ ]:
soil_moist_array = soil_moist.read(1)
affine = soil_moist.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, soil_moist_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "soil_moist"})

### Import pollutants

In [ ]:
#From S5P
no2_path = absolutepath + '/temp/no2.tif'
so2_path = absolutepath + '/temp/so2.tif'
aod55_path = absolutepath + '/temp/aod_055.tif'
aod47_path = absolutepath + '/temp/aod_047.tif'
uvai_path = absolutepath + '/temp/uvai.tif'
co_path = absolutepath + '/temp/co.tif'
form_path = absolutepath + '/temp/form.tif'

no2 = rio.open(no2_path)
so2 = rio.open(so2_path)
aod55 = rio.open(aod55_path)
aod47 = rio.open(aod47_path)
uvai = rio.open(uvai_path)
co = rio.open(co_path)
form = rio.open(form_path)

#From CAMS
nh3_path = absolutepath + '/temp/nh3_cams.nc'
no_path = absolutepath + '/temp/no_cams.nc'
co_path = absolutepath + '/temp/co_cams.nc'
no2_cams_path = absolutepath + '/temp/no2_cams.nc'
dust_path = absolutepath + '/temp/dust_cams.nc'
pm10_path = absolutepath + '/temp/pm10_cams.nc'
pm25_path = absolutepath + '/temp/pm25_cams.nc'
nmvocs_path = absolutepath + '/temp/nmvocs_cams.nc'
so2_cams_path = absolutepath + '/temp/so2_cams.nc'

nh3 = rioxarray.open_rasterio(nh3_path)
no = rioxarray.open_rasterio(no_path)
co = rioxarray.open_rasterio(co_path)
no2_cams = rioxarray.open_rasterio(no2_cams_path)
dust = rioxarray.open_rasterio(dust_path)
pm10 = rioxarray.open_rasterio(pm10_path)
pm25 = rioxarray.open_rasterio(pm25_path)
nmvocs = rioxarray.open_rasterio(nmvocs_path)
so2_cams = rioxarray.open_rasterio(so2_cams_path)

#Convert ammonia to .tif
nh3.rio.to_raster(absolutepath + "/temp/nh3_cams.tif")
nh3_path = absolutepath + '/temp/nh3_cams.tif'
nh3 = rio.open(nh3_path)
#Convert NO to .tif
no.rio.to_raster(absolutepath + "/temp/no_cams.tif")
no_path = absolutepath + '/temp/no_cams.tif'
no = rio.open(no_path)
#Convert CO to .tif
co.rio.to_raster(absolutepath + "/temp/co_cams.tif")
co_path = absolutepath + '/temp/co_cams.tif'
co = rio.open(co_path)
#Convert dust to .tif
dust.rio.to_raster(absolutepath + "/temp/dust_cams.tif")
dust_path = absolutepath + '/temp/dust_cams.tif'
dust = rio.open(dust_path)
#Convert pm10 to .tif
pm10.rio.to_raster(absolutepath + "/temp/pm10_cams.tif")
pm10_path = absolutepath + '/temp/pm10_cams.tif'
pm10 = rio.open(pm10_path)
#Convert pm25 to .tif
pm25.rio.to_raster(absolutepath + "/temp/pm25_cams.tif")
pm25_path = absolutepath + '/temp/pm25_cams.tif'
pm25 = rio.open(pm25_path)
#Convert NO2 to .tif
no2_cams.rio.to_raster(absolutepath + "/temp/no2_cams.tif")
no2_cams_path = absolutepath + '/temp/no2_cams.tif'
no2_cams = rio.open(no2_cams_path)
#Convert NMVOCs to .tif
nmvocs.rio.to_raster(absolutepath + "/temp/nmvocs_cams.tif")
nmvocs_path = absolutepath + '/temp/nmvocs_cams.tif'
nmvocs = rio.open(nmvocs_path)
#Convert SO2 to .tif
so2_cams.rio.to_raster(absolutepath + "/temp/so2_cams.tif")
so2_cams_path = absolutepath + '/temp/so2_cams.tif'
so2_cams = rio.open(so2_cams_path)

In [ ]:
no2_array = no2.read(1)
affine = no2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no2_s5p"})

In [ ]:
so2_array = so2.read(1)
affine = so2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, so2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "so2_s5p"})

In [ ]:
aod55_array = aod55.read(1)
affine = aod55.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aod55_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aod_055"})

In [ ]:
aod47_array = aod47.read(1)
affine = aod47.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aod47_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aod_047"})

In [ ]:
uvai_array = uvai.read(1)
affine = uvai.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, uvai_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "uvai"})

In [ ]:
co_array = co.read(1)
affine = co.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, co_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "co_s5p"})

In [ ]:
form_array = form.read(1)
affine = form.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, form_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "form_s5p"})

In [ ]:
nh3_array = nh3.read(1)
affine = nh3.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, nh3_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "amm_cams"})

In [ ]:
no_array = no.read(1)
affine = no.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no_cams"})

In [ ]:
co_array = co.read(1)
affine = co.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, co_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "co_cams"})

In [ ]:
dust_array = dust.read(1)
affine = dust.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, dust_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "dust_cams"})

In [ ]:
pm10_array = pm10.read(1)
affine = pm10.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pm10_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm10_cams"})

In [ ]:
pm25_array = pm25.read(1)
affine = pm25.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pm25_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm25_cams"})

In [ ]:
no2_cams_array = no2_cams.read(1)
affine = no2_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no2_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no2_cams"})

In [ ]:
nmvocs_array = nmvocs.read(1)
affine = nmvocs.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, nmvocs_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "nmvocs_cams"})

In [ ]:
so2_cams_array = so2_cams.read(1)
affine = so2_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, so2_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "so2_cams"})

### [Gridded Population of the World - GPW](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-rev11)<br>
To provide estimates of population density for the year 2020, based on counts consistent with national censuses and population registers, as raster data to facilitate data integration.
Input reference system EPSG: 4326

In [ ]:
pop_path = absolutepath + '/population/population.tif'
pop = rio.open(pop_path)

In [ ]:
pop_array = pop.read(1)
pop_array[pop_array<0]=np.nan
affine = pop.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pop_array, affine=affine, nodata=np.nan, stats=['sum'])), how='left')
grid = grid.rename(columns={"sum": "pop"})

 - - -

### [Road Infrastructures - Geoportale Lombardia (DBTR 2019)](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B17D4656F-2E9D-4951-9DC1-4AD32C0959B1%7D): 

**Point layers** considered:
1. Intersection between primary roads including highways
2. Intersection between primary and secondary roads
3. Intersection between secondary roads

Input reference system EPSG: 4326

In [ ]:
int_prim_path = absolutepath + '/road_infrastructures/inters_highway_prim_road.gpkg'
int_prim_sec_path = absolutepath + '/road_infrastructures/inters_prim_sec_road.gpkg'
int_sec_path = absolutepath + '/road_infrastructures/inters_sec_road.gpkg'

grid = grid.to_crs(32632)

In [ ]:
int_prim = gpd.read_file(int_prim_path).to_crs(32632)
int_prim_sec = gpd.read_file(int_prim_sec_path).to_crs(32632)
int_sec = gpd.read_file(int_sec_path).to_crs(32632)

df_dict = {'int_prim':int_prim,
          'int_prim_sec':int_prim_sec, 'int_sec': int_sec}


for key in df_dict:
    poor_points = df_dict[key][['OBJECTID','geometry']]
    sjoined = gpd.sjoin(poor_points, grid)
    df_count = pd.DataFrame(sjoined.groupby('index_right').size()) 
    grid_join = grid.join(df_count)
    grid[key] = grid_join[0]

**Line layers** considered:
1. Highways
2. Primary roads
3. Secondary roads

Input reference system EPSG: 4326

In [ ]:
highway_path = absolutepath + '/road_infrastructures/highway.gpkg'
prim_road_path = absolutepath + '/road_infrastructures/prim_road.gpkg'
sec_road_path = absolutepath + '/road_infrastructures/sec_road.gpkg'

It is required to convert to a cartographic reference system EPSG:32632 to calculate distances.

In [ ]:
highway = gpd.read_file(highway_path).to_crs(32632)
prim_road = gpd.read_file(prim_road_path).to_crs(32632)
sec_road = gpd.read_file(sec_road_path).to_crs(32632)

In [ ]:
df_dict = {'highway':highway, 'prim_road':prim_road, 'sec_road':sec_road}

for key in df_dict:
    grid[key] = np.nan
    poor_lines = df_dict[key][['geodb_oid','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_lines, mask) 
        l = clip.geometry.length.sum()
        grid[key].iloc[index] = l*m_to_km
    print(key)

 - - -

### Farms
Vector file obtained from DUSAF 2018 (features with cod. 12112 = "Insediamenti produttivi agricoli
Sono compresi in questa classe gli edifici utilizzati per le attività produttive del settore primario, come capannoni, rimesse per macchine agricole, fienili, stalle, silos, ecc, unitamente agli spazi accessori. Quando tali edifici sono presenti insieme a quelli residenziali configurando un aggregato rurale, se le due tipologie non risultano separabili in modo evidente si classifica tutto il nucleo come cascina (11231)").

In [ ]:
farms_path = absolutepath + '/farms/farms_dissolve.gpkg'
farms = gpd.read_file(farms_path)

In [ ]:
df_dict2 = {'farms':farms}

In [ ]:
for key in df_dict2:
    grid[key] = np.nan
    poor_poly = df_dict2[key][['COD_TOT','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_poly, mask) 
        a = clip.geometry.area.sum()
        grid[key].iloc[index] = a*m2_to_km2
    print(key)

In [ ]:
grid = grid.to_crs(4326)

---

### Soil and Vegetation ??
Reference system EPSG:4326

In [ ]:
# soil_path = absolutepath + '/soil/soil.tiff'

- 1 Argillosa
- 2 Argillosa limosa
- 3 Argillosa sabbiosa
- 4 Franca
- 5 Franco argillosa
- 6 Franco limosa
- 7 Franco limosa argillosa
- 8 Franco sabbiosa
- 9 Franco sabbiosa argillosa
- 10 Sabbiosa
- 11 Sabbioso franca


- - - 

# Import air quality and meteo stations 

Meteo station:

In [ ]:
temp_st_path = absolutepath + '/temp/temp_st.gpkg'
prec_st_path = absolutepath + '/temp/prec_st.gpkg'
air_hum_st_path = absolutepath + '/temp/air_hum_st.gpkg'
wind_dir_st_path = absolutepath + '/temp/wind_dir_st.gpkg'
wind_speed_st_path = absolutepath + '/temp/wind_speed_st.gpkg'
rad_glob_st_path = absolutepath + '/temp/rad_glob_st.gpkg'

In [ ]:
temp_st = gpd.read_file(temp_st_path)
prec_st = gpd.read_file(prec_st_path)
air_hum_st = gpd.read_file(air_hum_st_path)
wind_dir_st = gpd.read_file(wind_dir_st_path)
wind_speed_st = gpd.read_file(wind_speed_st_path)
rad_glob_st = gpd.read_file(rad_glob_st_path)

In [ ]:
df_dict = {'temp_st':temp_st,
          'prec_st':prec_st, 'air_hum_st': air_hum_st, 'wind_dir_st':wind_dir_st, 'wind_speed_st':wind_speed_st,
          'rad_glob_st':rad_glob_st}

In [ ]:
for key in df_dict:
    grid[key] = np.nan
    poor_points = df_dict[key][['idsensore','valore','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.valore.mean()
        grid[key].iloc[index] = m
    print(key)

Air quality station:

In [ ]:
pm25_st_path = absolutepath + '/temp/pm25_st.gpkg'
nox_st_path = absolutepath + '/temp/nox_st.gpkg'
no2_st_path = absolutepath + '/temp/no2_st.gpkg'
amm_st_path = absolutepath + '/temp/amm_st.gpkg'
so2_st_path = absolutepath + '/temp/so2_st.gpkg'
pm10_st_path = absolutepath + '/temp/pm10_st.gpkg'
co_st_path = absolutepath + '/temp/co_st.gpkg'

In [ ]:
pm25_st = gpd.read_file(pm25_st_path)
nox_st = gpd.read_file(nox_st_path)
no2_st = gpd.read_file(no2_st_path)
amm_st = gpd.read_file(amm_st_path)
so2_st = gpd.read_file(so2_st_path)
pm10_st = gpd.read_file(pm10_st_path)
co_st = gpd.read_file(co_st_path)

In [ ]:
df_dict = {'pm25_st':pm25_st,
          'nox_st':nox_st, 'no2_st': no2_st, 'amm_st':amm_st, 'so2_st':so2_st, 'pm10_st':pm10_st, 'co_st':co_st}

In [ ]:
for key in df_dict:
    grid[key] = 0
    poor_points = df_dict[key][['idsensore','valore','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.valore.mean()
        grid[key].iloc[index] = m
    print(key)

In [ ]:
grid

In [ ]:
grid.to_crs(4326).to_file("grid_prova0_1.gpkg", driver="GPKG")

In [ ]:
correl = grid.corr(method ='pearson')

In [ ]:
correl.to_csv(absolutepath+'/out.csv')